# Overview

The goal of this notebook is to explain a workflow for:
1. Setting up a Redshift Cluster for Spectrum
2. Using Glue Catalog crawler to define a schema for ETL output
3. Linking this catalog to Spectrum to query data _in situ_

Step 1 can be omitted if there is already a Spectrum cluster built. You can simply just create the glue catalog and create the external cluster

# Create Redhshift Role

1. Navigate to [IAM Roles page](https://console.aws.amazon.com/iam/home#/roles)
2. Create a new Role with a meaningful name like `<project_name>_spectrum_role`
    * This role should have the following policies: 
        - AmazonS3ReadOnlyAccess
        - AWSGlueConsoleFullAccess
3. After creating this role, it will be assigned a **Role ARN** (Amazon Resource Name). Make note of this ARN, it will be used later

# Create Redshift Cluster

1. Navigate to [Redshift Cluster Page](https://console.aws.amazon.com/redshiftv2/home?region=us-east-1#dashboard)
2. In upper Right corner, select the region for your cluster
3. Click "Create Cluster"
4. Configure your cluster
    * Name
    * Node Type. For interactively using Query Editor(_user must have `AmazonRedshiftQueryEditor`, and `AmazonRedshiftReadOnlyAccess` policies active_) select one of the following:
        - DC1.8xlarge
        - DC2.large
        - DC2.8xlarge
        - DS2.8xlarge
        - RA3.xlplus
        - RA3.4xlarge
        - RA3.16xlarge  
        
    * Number of Nodes  
5. Configure the Database credentials. Make note of these, you need them to log in to query editor.
    * Make a master user name
    * Make a user password
    

# Create Glue Catalog

In order to query the data, there must be a defined schema, which can be done using Glue Crawlers.
1. Go to [AWS Glue Console](https://console.aws.amazon.com/glue/)
2. Create a crawler.
    * Create a new database 
    * Then choose the data store which is the root of where all your desired tables are located in S3.
3. Run the Crawler and confirm that the schema looks accurate.

# Add Glue Database to Spectrum

This is added via DDL in the query editor
1. Go to your [Query Editor](https://console.aws.amazon.com/redshiftv2/home?region=us-east-1#query-editor:)
    * The first time you do this you may have to 
2. Run the following command to attach the external schema:

    ```sql
    CREATE EXTERNAL SCHEMA <desired_spectrum_database_name>
    FROM data catalog database '<glue_database_name>' 
        iam_role '<arn_id_from_section_2>'
    CREATE EXTERNAL DATABASE IF NOT EXISTS
    ```

# DONE 

You can now query the database in Spectrum, if schema changes occur in the parquet data, then you can rerun AWS Glue to recatalog the data and Redshift will stay in sync with this.

# Resources

* https://docs.aws.amazon.com/redshift/latest/gsg/getting-started.html
* https://docs.aws.amazon.com/redshift/latest/dg/c-getting-started-using-spectrum.html